In [0]:
# raw_data_path = "dbfs:/mnt/upgrade_to_uc/raw/"
raw_data_path = "/Volumes/main/uc_upgrade/volume"



checkpointPath = "abfss://container@databrickssqlmetastore.dfs.core.windows.net/raw/"

# catalog = "hive_metastore"
catalog = "main"

In [0]:
gen2_credential = dbutils.secrets.get("Credentials", "Gen2")

In [0]:
storage_account_name = 'databrickssqlmetastore'
storage_account_access_key = gen2_credential
spark.conf.set('fs.azure.account.key.' + storage_account_name + '.blob.core.windows.net', storage_account_access_key)

spark._jsc.hadoopConfiguration().set("fs.azure.account.key.databrickssqlmetastore.dfs.core.windows.net", gen2_credential)


In [0]:
from pyspark.sql.types import StructType

schema = StructType().add("summary", "string").add("SourceName", "string").add("PlanId", "string").add("IndivisualRate", "string").add("Age", "string").add("RatingAreaId", "string").add("Tobacco", "string").add("MetalLevel", "string").add("PlanType", "string").add("DentalOnlyPlan", "string").add("MedicalDrugDeductiblesIntegrated", "string").add("PlanMarketingName", "string").add("TEHBCombInnOonIndividualMOOP", "string").add("WellnessProgramOffered", "string").add("InpatientCopaymentMaximumDays", "string").add("AVCalculatorOutputNumber", "string").add("CSRVariationType", "string").add("DiseaseManagementProgramsOffered", "string").add("IssuerActuarialValue", "string").add("prediction", "string").add("std_diff", "string")

(spark.readStream.format("cloudFiles")
 .schema(schema)
 .option("cloudFiles.format", "csv")
 .option("header", "false")
 .option("cloudFiles.schemaLocation", raw_data_path+"/autoloader/bronze/")
 .load(raw_data_path)
 .writeStream
 .option("checkpointLocation", raw_data_path+"/autoloader/bronze/")
 .toTable(catalog+".uc_upgrade.raw_bronze")
 )